In [3]:
"""
This python script will preprocess the data images.
It will import the images from center, left, and right camera
and turn it into numpy arrays. These numpy arrays will be splitted
into train and validation sets and saved as pickle.
"""
import argparse
import os
import sys
import csv
import base64
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

### Paths to folder and label
folder_path = "C:/Users/vikas/Desktop/behaviourial_cloning/training_data"
label_path = "{}/driving_log.csv".format(folder_path)


### Import data
data = []
with open(label_path) as F:
    reader = csv.reader(F)
    for i in reader:
        data.append(i) 

print("data imported starts here ")

### size of the data
data_size = len(data)
print("data size is:", data_size)

### Emtpy generators for feature and labels
features = ()
labels = ()

### This function will resize the images from front, left and
### right camera to 18 x 80 and turn them into lists.
### The length of the each list will be 18 x 80 = 1440
### j = 0,1,2 corresponds to center, left, right
def load_image(data_line, j):
    img = plt.imread(data_line[j].strip())[65:135:4,0:-1:4,0]
    lis = img.flatten().tolist()
    return lis

data = data[:100]

# For each item in data, convert camera images to single list
# and save them into features list.
for i in tqdm(range(int(len(data))), unit='items'):
    for j in range(3):
        features += (load_image(data[i],j),)

item_num = len(features)
print("features size", item_num)

# A single list will be convert back to the original image shapes.
# Each list contains 3 images so the shape of the result will be
# 54 x 80 where 3 images aligned vertically.
features = np.array(features).reshape(item_num, 18, 80, 1)
print("features shape", features.shape)

### Save labels    
for i in tqdm(range(int(len(data))), unit='items'):
    for j in range(3):
        labels += (float(data[i][3]),)

labels = np.array(labels)

print("features:", features.shape)
print("labels:", labels.shape)

from sklearn.cross_validation import train_test_split

# Get randomized datasets for training and test
X_train, X_test, y_train, y_test = train_test_split(
    features,
    labels,
    test_size=0.10,
    random_state=832289)

# Get randomized datasets for training and validation
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train,
    y_train,
    test_size=0.25,
    random_state=832289)

# Print out shapes of new arrays
train_size = X_train.shape[0]
test_size = X_test.shape[0]
valid_size = X_valid.shape[0]
input_shape = X_train.shape[1:]
features_count = X_train.shape[1]*X_train.shape[2]*X_train.shape[3]

print("train size:", train_size)
print("valid size:", valid_size)
print("test size:", test_size)
print("input_shape:", input_shape)
print("features count:", features_count)

import pickle

# Save the data for easy access
pickle_file = 'C:/Users/vikas/Desktop/behaviourial_cloning/examples/carnd.pickle'
stop = False

while not stop:
    if not os.path.isfile(pickle_file):
        print('Saving data to pickle file...')
        try:
            with open(pickle_file, 'wb') as pfile:
                pickle.dump(
                    {
                        'train_dataset': X_train,
                        'train_labels': y_train,
                        'valid_dataset': X_valid,
                        'valid_labels': y_valid,
                        'test_dataset': X_test,
                        'test_labels': y_test,
                    },
                    pfile, pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to', pickle_file, ':', e)
            raise

        print('Data cached in pickle file.')
        stop = True
    else:
        print("Please use a different file name other than carnd.pickle")
        pickle_file = input("Enter: ")


data imported starts here 
data size is: 438


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 26.47items/s]


features size 300
features shape (300, 18, 80, 1)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 39964.78items/s]


features: (300, 18, 80, 1)
labels: (300,)
train size: 202
valid size: 68
test size: 30
input_shape: (18, 80, 1)
features count: 1440
Saving data to pickle file...
Data cached in pickle file.


In [ ]:
# Load the modules
import pickle
import math
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
# Import keras deep learning libraries
import json
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
from keras import backend as K

# Reload the data
pickle_file = 'C:/Users/vikas/Desktop/behaviourial_cloning/examples/carnd.pickle'
with open(pickle_file, 'rb') as f:
    pickle_data = pickle.load(f)
    X_train = pickle_data['train_dataset']
    y_train = pickle_data['train_labels']
    X_valid = pickle_data['valid_dataset']
    y_valid = pickle_data['valid_labels']
    X_test = pickle_data['test_dataset']
    y_test = pickle_data['test_labels']
    del pickle_data  # Free up memory

# Print shapes of arrays that are imported
print('Data and modules loaded.')
print("train_features size:", X_train.shape)
print("train_labels size:", y_train.shape)
print("valid_features size:", X_valid.shape)
print("valid_labels size:", y_valid.shape)
print("test_features size:", X_test.shape)
print("test_labels size:", y_test.shape)

# the data, shuffled and split between train and test sets
X_train = X_train.astype('float32')
X_valid = X_valid.astype('float32')
X_test  = X_test.astype('float32')
X_train /= 255
X_valid /= 255
X_test  /= 255
X_train -= 0.5
X_valid -= 0.5
X_test  -= 0.5

# This is the shape of the image
input_shape = X_train.shape[1:]
print(input_shape, 'input shape')

# Set the parameters and print out the summary of the model
np.random.seed(1337)  # for reproducibility

batch_size = 64 # The lower the better
nb_classes = 1 # The output is a single digit: a steering angle
nb_epoch = 10 # The higher the better

# import model and wieghts if exists
try:
	with open('model.json', 'r') as jfile:
	    model = model_from_json(json.load(jfile))

	# Use adam and mean squared error for training
	model.compile("adam", "mse")

	# import weights
	model.load_weights('model.h5')

	print("Imported model and weights")

# If the model and weights do not exist, create a new model
except:
	# If model and weights do not exist in the local folder,
	# initiate a model

	# number of convolutional filters to use
	nb_filters1 = 16
	nb_filters2 = 8
	nb_filters3 = 4
	nb_filters4 = 2

	# size of pooling area for max pooling
	pool_size = (2, 2)

	# convolution kernel size
	kernel_size = (3, 3)

	# Initiating the model
	model = Sequential()

	# Starting with the convolutional layer
	# The first layer will turn 1 channel into 16 channels
	model.add(Convolution2D(nb_filters1, kernel_size[0], kernel_size[1],
	                        border_mode='valid',
	                        input_shape=input_shape))
	# Applying ReLU
	model.add(Activation('relu'))
	# The second conv layer will convert 16 channels into 8 channels
	model.add(Convolution2D(nb_filters2, kernel_size[0], kernel_size[1]))
	# Applying ReLU
	model.add(Activation('relu'))
	# The second conv layer will convert 8 channels into 4 channels
	model.add(Convolution2D(nb_filters3, kernel_size[0], kernel_size[1]))
	# Applying ReLU
	model.add(Activation('relu'))
	# The second conv layer will convert 4 channels into 2 channels
	model.add(Convolution2D(nb_filters4, kernel_size[0], kernel_size[1]))
	# Applying ReLU
	model.add(Activation('relu'))
	# Apply Max Pooling for each 2 x 2 pixels
	model.add(MaxPooling2D(pool_size=pool_size))
	# Apply dropout of 25%
	model.add(Dropout(0.25))

	# Flatten the matrix. The input has size of 360
	model.add(Flatten())
	# Input 360 Output 16
	model.add(Dense(16))
	# Applying ReLU
	model.add(Activation('relu'))
	# Input 16 Output 16
	model.add(Dense(16))
	# Applying ReLU
	model.add(Activation('relu'))
	# Input 16 Output 16
	model.add(Dense(16))
	# Applying ReLU
	model.add(Activation('relu'))
	# Apply dropout of 50%
	model.add(Dropout(0.5))
	# Input 16 Output 1
	model.add(Dense(nb_classes))

# Print out summary of the model
model.summary()

# Compile model using Adam optimizer 
# and loss computed by mean squared error
model.compile(loss='mean_squared_error',
              optimizer=Adam(),
              metrics=['accuracy'])

### Model training
history = model.fit(X_train, y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_valid, y_valid))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

import json
import os
import h5py

# Save the model.
# If the model.json file already exists in the local file,
# warn the user to make sure if user wants to overwrite the model.
if 'model.json' in os.listdir():
	print("The file already exists")
	print("Want to overwite? y or n")
	user_input = input()

	if user_input == "y":
		# Save model as json file
		json_string = model.to_json()

		with open('C:/Users/vikas/Desktop/behaviourial_cloning/examples/model.json', 'w') as outfile:
			json.dump(json_string, outfile)

			# save weights
			model.save_weights('.C:/Users/vikas/Desktop/behaviourial_cloning/examples/model.h5')
			print("Overwrite Successful")
	else:
		print("the model is not saved")
else:
	# Save model as json file
	json_string = model.to_json()

	with open('model.json', 'w') as outfile:
		json.dump(json_string, outfile)

		# save weights
		model.save_weights('.C:/Users/vikas/Desktop/behaviourial_cloning/examples/model.h5')
		print("Saved")

Using TensorFlow backend.


Data and modules loaded.
train_features size: (202, 18, 80, 1)
train_labels size: (202,)
valid_features size: (68, 18, 80, 1)
valid_labels size: (68,)
test_features size: (30, 18, 80, 1)
test_labels size: (30,)
(18, 80, 1) input shape


C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel\__main__.py:97: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), padding="valid", input_shape=(18, 80, 1...)`
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel\__main__.py:101: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3))`
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel\__main__.py:105: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(4, (3, 3))`
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel\__main__.py:109: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(2, (3, 3))`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 78, 16)        160       
_________________________________________________________________
activation_1 (Activation)    (None, 16, 78, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 76, 8)         1160      
_________________________________________________________________
activation_2 (Activation)    (None, 14, 76, 8)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 74, 4)         292       
_________________________________________________________________
activation_3 (Activation)    (None, 12, 74, 4)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 72, 2)         74        
__________

C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 202 samples, validate on 68 samples
Epoch 1/10
